<a href="https://colab.research.google.com/github/bartlomiejmiszewski/Nauka/blob/main/01_Szkolenie_preprocesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Szkolenie preprocesing

### scikit-learn
Strona biblioteki: [https://scikit-learn.org](https://scikit-learn.org)  

Dokumentacja/User Guide: [https://scikit-learn.org/stable/user_guide.html](https://scikit-learn.org/stable/user_guide.html)

### Ćwiczenia przy użyciu poniższych bibliotek:

*   from sklearn.preprocessing import LabelEncoder
*   from sklearn.preprocessing import OneHotEncoder
*   from sklearn.preprocessing import scale
*   from sklearn.preprocessing import StandardScaler

oraz pd.get_dummies()


### Preprocessing danych spis treści ćwiczeń:



1.   [Import bibliotek](#0)
2.   [Wygenerowanie danych](#1)
3.   [Utworzenie kopii danych](#2)
4.   [Zmiana typu danych i wstępna eksploracja](#3)
5.   [LabelEncoder](#4)
6.   [OneHotEncoder](#5)
7.   [pd.get_dummies()](#6)
8.   [Standaryzacja - StandardScaler](#7)
9.   [Przygotowanie danych do modelu - podsumowanie](#8)




### <a name='0'></a> 1. Import bibliotek

In [376]:
import pandas as pd
import numpy as np
import sklearn

### <a name='1'></a> 2. Wygenerowanie danych

In [377]:
dane = {
    'size' : ['XL', 'L', 'M', 'L', 'M'],
    'color' : ['red', 'green', 'blue', 'green', 'red'],
    'gender': ['female', 'male', 'male', 'female', 'female'],
    'price': [199.0, 89.0, 99.0, 129.0, 79.0],
    'weight': [500, 450, 300, 380, 410],
    'bought': ['yes', 'no', 'yes', 'no', 'yes']
}

In [378]:
df_raw = pd.DataFrame(data=dane)
df_raw

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


### <a name='2'></a> 3. Utworzenie kopii danych

In [379]:
df = df_raw.copy()

df.info()
#df.shape
#df.describe().T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   5 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  5 non-null      int64  
 5   bought  5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 368.0+ bytes


### <a name='3'></a> 4. Zmiana typu danych i wstępna eksploracja

In [380]:
#zamiana kolumn typu object na kolumny typu category
for col in ['size', 'color', 'gender', 'bought']:
  df[col] = df[col].astype('category')

In [381]:
#zamiana kolumny typu int na kolumnę typu float
df['weight'] = df['weight'].astype('float')

In [382]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   size    5 non-null      category
 1   color   5 non-null      category
 2   gender  5 non-null      category
 3   price   5 non-null      float64 
 4   weight  5 non-null      float64 
 5   bought  5 non-null      category
dtypes: category(4), float64(2)
memory usage: 628.0 bytes


In [383]:
#df.describe().T
df.describe(include=['category']).T

,count,unique,top,freq
size,5,3,M,2
color,5,3,red,2
gender,5,2,female,3
bought,5,2,yes,3


### <a name='4'></a> 5. LabelEncoder

In [384]:
#ten rodzaj encodowania zalecany jest tylko dla zmiennej objaśnianej nazywanej często zmienną "y"

In [385]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() #tworzenie instancji klasy LabelEncoder i przypisanie do zmiennej le
#le.fit(df['bought']) #metoda dopasowująca encoder do zmiennej bought
#le.transform(df['bought']) #metoda mapowania do zmiennej bought w tabeli df
le.fit_transform(df['bought']) #skrócony zapis dwóch powyższych metod

array([1, 0, 1, 0, 1])

In [386]:
le.classes_ #atrybut listujący zmapowane kategorie/klasy

array(['no', 'yes'], dtype=object)

In [387]:
df['bought'] = le.fit_transform(df['bought']) #przypisanie zmian do tabelki w kolumnie bought
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,green,male,89.0,450.0,0
2,M,blue,male,99.0,300.0,1
3,L,green,female,129.0,380.0,0
4,M,red,female,79.0,410.0,1


In [388]:
#istnieje metoda odwracająca poczynione zmiany
le.inverse_transform(df['bought'])

array(['yes', 'no', 'yes', 'no', 'yes'], dtype=object)

In [389]:
df['bought'] = le.inverse_transform(df['bought'])
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,male,99.0,300.0,yes
3,L,green,female,129.0,380.0,no
4,M,red,female,79.0,410.0,yes


### <a name='5'></a> 6. OneHotEncoder

In [390]:
#jak się okaże w następnym punkcie, być może lepszą metodą niz OneHotEncoder jest pd.get_dummies()

In [391]:
#OneHotEncoder jes zalecany z kolei do encodowania zmiennych objaśniających nazywanych często "x"

In [392]:
from sklearn.preprocessing import OneHotEncoder

In [393]:
ohe = OneHotEncoder(sparse=False) #gdy parametr sparse jest ustwiony domyślnie to w wyniku jest pokazywana tylko macierz rzadka
ohe.fit_transform(df[['size']]) #trzeba pamiętać aby nazwę kolumny umieścić w jeszcze jednym nawiasie kwadratowym

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [394]:
ohe.categories_ #kategorie, które encodowano

[array(['L', 'M', 'XL'], dtype=object)]

In [395]:
#należy koniecznie pamiętać aby parametr drop ustawić na 'first' aby pozbyc się liniowej zależności pomiezy zmiennymi !!!!!!!!!!
#jeśli w zbiorze danych jest n cech to po encodowniau powinno być ich n-1
ohe = OneHotEncoder(drop='first', sparse=False) 
ohe.fit_transform(df[['size']])

array([[0., 1.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 0.]])

In [396]:
#df['size'] = ohe.fit_transform(df[['size']])
#df

### <a name='6'></a> 7. get_dummies()

In [397]:
# chyba najlepsza metoda do encodowania danych kategorycznych

In [398]:
#pd.get_dummies(data=df) #<-tak nie robić :D
pd.get_dummies(data=df, drop_first=True) #podobnie jak w OneHotEncoder pozbywwamy sie pierwszej kolumny za pomocą parametru drop_first

#pd.get_dummies(data=df, drop_first=True, columns=['size']) #można również encodować niektóre kolumny

,price,weight,size_M,size_XL,color_green,color_red,gender_male,bought_yes
0,199.0,500.0,0,1,0,1,0,1
1,89.0,450.0,0,0,1,0,1,0
2,99.0,300.0,1,0,0,0,1,1
3,129.0,380.0,0,0,1,0,0,0
4,79.0,410.0,1,0,0,1,0,1


### <a name='7'></a> 8. Standaryzacja - StandardScaler

In [399]:
#standaryzuje sie dane numeryczne aby poszczególne zmienne miały równą "moc"
#na przykład waga ma większe wartości niż cena przez co waga może mocniej wpływać na wyniki w modelu niż cena
#dlatego najlepiej i najbezpieczniej jest sobie wystandaryzować zmienne

In [400]:
from sklearn.preprocessing import StandardScaler

ssc = StandardScaler()
#ssc.fit(df[['price']])
#ssc.transform(df[['price']])
ssc.fit_transform(df[['price']])

array([[ 1.84506242],
       [-0.69189841],
       [-0.4612656 ],
       [ 0.2306328 ],
       [-0.92253121]])

In [401]:
#ssc.fit_transform(df[['price', 'weight']])

In [402]:
df.describe().columns

Index(['price', 'weight'], dtype='object')

In [403]:
#fajny spsób na zarzucenie pętli na standaryzację wszystkich kolumn numerycznych
#columns = ['price', 'weight']
for col in df.describe().columns:
  df[col] = ssc.fit_transform(df[[col]])

In [404]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,1.845062,1.366002,yes
1,L,green,male,-0.691898,0.623610,no
2,M,blue,male,-0.461266,-1.603567,yes
3,L,green,female,0.230633,-0.415740,no
4,M,red,female,-0.922531,0.029696,yes


### <a name='8'></a> 9. Przygotowanie danych do modelu - podsumowanie

In [405]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,1.845062,1.366002,yes
1,L,green,male,-0.691898,0.623610,no
2,M,blue,male,-0.461266,-1.603567,yes
3,L,green,female,0.230633,-0.415740,no
4,M,red,female,-0.922531,0.029696,yes


In [406]:
df = df_raw.copy()
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


In [407]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   5 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  5 non-null      int64  
 5   bought  5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 368.0+ bytes


In [408]:
#import numpy as np
#import pandas as pd
#from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.preprocessing import StandardScaler # import bibliotek został wykonany wcześniej

#najpierw powinno być chyba uzupełnianie braków?


#optymalizacja typów zmiennych numerycznych
for col in df.describe().columns:
  df[col] = df[col].astype('float')


#optymalizacja typów zmiennych tekstowych
#for colCat in df.describe(include='object').columns: # coś tu nie działa
colCat = ['size', 'color', 'gender', 'bought']
for col in colCat:
  df[col] = df[col].astype('category')


#standaryzacja zmiennych numerycznych
scaler = StandardScaler()

for col in df.describe().columns:
  df[col] = scaler.fit_transform(df[[col]])


#encodowanie zmiennej objaśnianej "y"
lenc = LabelEncoder()
df['bought'] = lenc.fit_transform(df['bought'])


#encodowanie zmiennych objaśniających "X"
columns_X = ['size', 'color', 'gender']
df = pd.get_dummies(data=df, drop_first=True, columns=columns_X)

In [409]:
df

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,1.845062,1.366002,1,0,1,0,1,0
1,-0.691898,0.623610,0,0,0,1,0,1
2,-0.461266,-1.603567,1,1,0,0,0,1
3,0.230633,-0.415740,0,0,0,1,0,0
4,-0.922531,0.029696,1,1,0,0,1,0
